# 🚀 Quellcode-Analyse einer Website 

<div class="alert alert-block alert-info"> <b> 🔔 Feinlernziel(e) dieses Kapitels</b></br>
   Sie können die Semantik der textangebenden html-Tags beschreiben und beschreiben, welche Tags zur Textextraktion verwendet werden. </br>
Sie können den Unterschied zwischen statischen und flexiblen Webinhalten benennen. 
</div>

## Hinweise zur Ausführung des Notebooks
Dieses Notebook kann auf unterschiedlichen Levels erarbeitet werden (siehe Abschnitt ["Technische Voraussetzungen"](../markdown/introduction_requirements)): 
1. Book-Only Mode
2. Cloud Mode: Dafür auf 🚀 klicken und z.B. in Colab ausführen.
3. Local Mode: Dafür auf Herunterladen ↓ klicken und ".ipynb" wählen. 

## Übersicht
Im Folgenden wird exemplarisch der HTML-Code der Website der Senatskanzlei Berlin auf seine Struktur hin untersucht und es wird eine strukturierte Methode zur Inhaltsextraktion entwickelt.

Dafür werden folgendene Schritte durchgeführt:
1. Abfragen des HTML-Codes 
2. Strukturiertes Parsen des HTML-Codes
4. Verlinkten Seiten nachgehen und parsen
5. Ergebnisse speichern

<details>
  <summary><b>Informationen zum Ausführen des Notebooks – Zum Ausklappen klicken ⬇️</b></summary>
  
<b>Voraussetzungen zur Ausführung des Jupyter Notebooks:</b>
<ul>
<li> Installieren der Bibliotheken </li>
</ul>
Zum Testen: Ausführen der Zelle "load libraries".</br>
Alle Zellen, die mit 🚀 gekennzeichnet sind, werden nur bei der Ausführung des Noteboos in Colab / JupyterHub bzw. lokal ausgeführt. 
</details>

In [ ]:
#  🚀 Install libraries 
! pip install requests beautifulsoup4 pandas

In [ ]:
# load libraries
from datetime import datetime

import requests
from bs4 import BeautifulSoup, Tag, Comment

## 1. HTML-Code der Website der Senatskanzlei abfragen


In [ ]:
# Set url to the URL of the senatskanzlei Berlin
url = "https://www.berlin.de/rbmskzl/"

In [ ]:
# Perform get request, save HTML in the variable response
response = requests.get(url)

# Check if the get request was successful
if response.status_code == 200:
    print("Die Abfrage des HTML-Codes war erfolgreich")
else:
    print(f"Die Abfrage war nicht erfolgreich! Der Fehlercode ist: {response.status_code}")

In [ ]:
# Get the HTML text from the reponse
html_text = response.text

# display the first 100 characters of the HTML
html_text[:100]

In [ ]:
soup = BeautifulSoup(html_text)

## 2. Strukturiertes Parsen des HTML-Codes
1. Aufbau der Seite analysieren und Auswahl treffen
2. Top-News-Sektion parsen
3. Politische Themen parsen
4. Pressemitteilungen abfragen 

### 2.1 Aufbau der Seite analysieren
![berlin_de_top](berlin_de_top.png)

4 Sektionen
1. Top-Sektion: A -- könnte auch (so wie es aussieht) aus 2 Teilen bestehen
2. Oft gesucht: B
3. Politische Themen: C
4. Pressemitteilungen: D

Uns interessiert A, C und D

#### Struktur von A und C
* Überschrift, Text, Link in "Weitere Informationen"

#### Struktur von D
* Datum, Titel, Link in dem Titel

Strategie: 
1. Ist die visuelle Aufteilung in den Tags abgebildet?
  * Ist der Tag, der die Abschnitte unterteilt einmalig für den jeweiligen Abschnitt?
3. Welche Tags (mit Attribut) fassen die einzelnen Teile zusammen?
4. Wie sind die Inhalte in den einzelnen Teile strukturiert? Welche hierarchische Gliederung gibt es? 

### 2.2 A (Top-News-Sektion) parsen

#### 2.2.1 Ausfinding machen, ob es einen Tag gibt, dem die Inhalte der Sektion A untergeordnet sind. 

Das ist der Fall für den div-Container `<div>`  mit CSS class `'herounit-homepage herounit-homepage--default'`.
Mit der Library `beautifulsoup` kann der gesamte Container, also alle tags, die dem div-Container untergeordnet sind, extrahiert werden.

In [ ]:
# get all tags that are children of the div tag with matching CSS class
topdiv = soup.find("div", {"class": "herounit-homepage herounit-homepage--default"})

# print the content of the topdiv
print(topdiv.prettify())

#### 2.2.2 Titel extrahieren

Wir sehen, dass alle Titel unter `h2`-Tags stehen. Diese können wir im nächsten Schritt extrahieren. Wir gehen dabei von dem bereits extrahierten Top-Div aus und extrahieren nur `h2`-Tags, die diesem Tag untergeordnet sind. 

In [ ]:
topdiv_h2titles = topdiv.find_all('h2')

In [ ]:
# get all h2 content that is a child of the top div
topdiv_h2titles = topdiv.find_all('h2')

# retrieve the content and clean it
topdiv_h2titles = [entry.text.strip() for entry in topdiv_h2titles]

print(topdiv_h2titles)

#### 2.2.3 Kurzbeschreibungen extrahieren 

Wir sehen weiter, dass alle Kurzbeschreibungen als paragraphs `<p>` ausgezeichnet sind. Im Folgenden extrahieren wir alle Paragraphen und lassen uns das Ergebnis anzeigen.


In [ ]:
# get all paragraphs that are children of the top div
topdiv_texts =  topdiv.find_all('p')
topdiv_texts

Wir extrahieren zwar so alle Kurzbeschreibungen, unsere Liste beinhaltet allerdings auch die Beschreibung eines Bilds. Da sich das `class`-Attribut der Kurzbeschreibung von dem des Bilds unterscheidet, können wir durch das zusätzliche Abgleichen des Attributs eine Liste erstellen, in der nur die Kurzbeschreibungen vorhanden sind: 

In [ ]:
# get all short description from the p for which the attribute "class" equals "text"
topdiv_texts =  topdiv.find_all('p', {"class":"text"})

# retrieve the content and clean it
topdiv_texts = [entry.text.strip() for entry in topdiv_texts]

# print the extracted content
topdiv_texts

#### 2.2.4 Links extrahieren

Auf die gleiche Weise können wir alle Hyperlinks, die in `<a>`-Tags gespeichert sind extrahieren. Der Hyperlink selbst steht in dem Attribut `href`, dessen Wert wir gezielt abfragen.

In [ ]:
topdiv_links =  topdiv.find_all('a')
topdiv_links = [entry.get('href') for entry in topdiv_links]

# print the extracted links
topdiv_links

Wir sehen, dass die Links keine vollständigen URLs sind, da sie weder mit "www." noch mit "https://" anfangen. Diese Links nennen wir **relative URLs**. Sie verweisen auf Unterseiten der aktuellen Seite (die Startseite der Senatskanzlei), die Adresse der Unterseiten wird relativ zur aktuellen Seite angegeben.

Um aus den relativen URLs absolte URLs zu erzeugen, die auch durch Einfügen in die Adresszeile eines Browsers abrufbar sind, muss den URLs das Präfix der aktuellen Seite vorangestellt werden, in unserem Fall "https://www.berlin.de/".

In [ ]:
# create absolute URLs
def make_links_absolute(link_list, prefix="https://www.berlin.de"):
    absolute_links = []
    for link in link_list:
        if not link.startswith("https"):
            absolute_links.append(prefix + link)
        else:
            absolute_links.append(link)    
    return absolute_links

In [ ]:
topdiv_links = make_links_absolute(topdiv_links)
topdiv_links

#### Zusammenfügen der Daten 
* alle müssen die gleich Länge haben
* verlassen uns hier auf die Reihenfolge der Daten in den verschiedenen Listen

In [ ]:
len(topdiv_h2titles) == len(topdiv_texts) == len(topdiv_absolute_links)

In [ ]:
top_section_data = {"Titel": topdiv_h2titles,
                  "Text":topdiv_texts, 
                  "URL":topdiv_absolute_links,
                   "Datum": datetime.today().strftime('%d.%m.%Y') }

In [ ]:
top_section_data_df = pd.DataFrame(top_section_data)
top_section_data_df

### 2.3 Teil B (Politische Themen) parsen 

In [ ]:
sections = soup.findAll("section", {"class": "modul-multiteaser"})
print(len(sections))
print(sections[1].prettify())

In [ ]:
def process_single_section(section, prefix=""):
    content =  {"Titel":[], "Text":[]}
    titles_with_tags = section.find_all('h3')
    content["Titel"] = [entry.text.strip() for entry in titles_with_tags]
    
    section_ps = section.find_all('p', {"class":"text"})
    links = []
    for p in section_ps:
        content["Text"].append(p.find(string=True).strip())
        links.append(p.find('a').get('href'))
    content["URL"] = make_links_absolute(links)
    return content

In [ ]:
politische_themen = {"Titel":[], "Text":[], "URL":[]}
for section in sections:
    content = process_single_section(section)
    politische_themen.update(content)
politische_themen["Datum"] = datetime.today().strftime('%d.%m.%Y')

In [ ]:
politische_themen_df = pd.DataFrame(politische_themen)
politische_themen_df

### 2.4 Pressemitteilungen parsen

In [ ]:
pressemitteilungen_parsed = {}

pressemitteilungen = soup.find("ul", {"class":"list--tablelist ruler has-date"})
pressemitteilungen_parsed["Datum"] = [date.text for date in pressemitteilungen.findAll("div", {"class":"cell date"})]
pressemitteilungen_parsed["Text"] =  [a.text for a in pressemitteilungen.findAll("a")]

links = [a.get("href") for a in pressemitteilungen.findAll("a")]
pressemitteilungen_parsed["URL"] = make_links_absolute(links)


In [ ]:
pressemitteilungen_parsed_df = pd.DataFrame(pressemitteilungen_parsed)

### 2.5 Daten der Startseite zusammenfügen 

In [ ]:
startseite_df = pd.concat([top_section_data_df, politische_themen_df, pressemitteilungen_parsed_df])

In [ ]:
startseite_df

## 3. Links zu den Mitteilungen nachgehen 

### 3.1 Extraktion des Volltexts der verlinkten Websites 

In [ ]:
# https://stackoverflow.com/questions/1936466/how-to-scrape-only-visible-webpage-text-with-beautifulsoup

# Only get text that is visible on the website 
# Inlcudes ads, pointers to more info and so on 

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

def text_from_html(soup):
    texts = soup.findAll(string=True)
    visible_texts = filter(tag_visible, texts)  
    return [t.strip() for t in visible_texts if len(t) > 1]

In [ ]:
full_texts = []
for url in startseite_df.URL:
    response = requests.get(url)
    if response.status_code == 200:
        html_text = response.text
        soup = BeautifulSoup(html_text)
        text = text_from_html(soup)
        full_texts.append(text)

### 3.2 Dirty clean the full texts by setting a length threshold

In [ ]:
lenghts = [len(entry) for entry in full_texts[0]]
sorted(lenghts)[-5:]

In [ ]:
threshold = 100
cleaned_full_text = []
for entry in full_texts:
    cleaned = []
    for snippet in entry:
        if len(snippet) > threshold:
            cleaned.append(snippet)
    cleaned_full_text.append("\n".join(cleaned))

In [ ]:
startseite_df["Volltext"] = cleaned_full_text

In [ ]:
startseite_df

## 4. Ergebnisse speichern 

In [ ]:
date = datetime.today().strftime('%Y-%m-&d')
startseite_df.to_csv(f"{date}_Startseite_Senatskanzlei_Berlin.csv", index=False)